In [1]:
import os
import torch
from fastai.torch_imports import resnet34
from fastai.transforms import tfms_from_model
from fastai.conv_learner import ConvnetBuilder
from fastai.core import V, VV, to_np, is_listy
from fastai.dataset import FilesIndexArrayDataset
from fastai.dataloader import DataLoader
from fastai.model import get_prediction
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
sz = 224
bs = 128
model = resnet34
model_weight_path = 'data/ternonter/models/224_all_ter_train_aug.h5'

In [3]:
trn_tfms, val_tfms = tfms_from_model(model, sz)
preprocess = val_tfms
model = ConvnetBuilder(model, 2, False, False, pretrained=False).model
sd = torch.load(model_weight_path)
model.load_state_dict(sd)
# self.model = self.model.cpu()
model.eval();

In [4]:
data_dir = 'data/ternonter/test1/'
flist = os.listdir(data_dir)
ds = FilesIndexArrayDataset(flist, np.ones(len(flist)), preprocess, path=data_dir)
dl = DataLoader(ds, batch_size=bs)

In [5]:
def get_predictions(model, dl):
    res = []
    for *x,y in dl: 
        res.append([get_prediction(to_np(model(*VV(x)))),to_np(y)])
    preds,targ = zip(*res)
    return np.concatenate(preds)

In [6]:
%%time

preds = get_predictions(model, dl)

CPU times: user 20.2 s, sys: 14.9 s, total: 35.1 s
Wall time: 22.5 s


In [10]:
preds[:10,:]

array([[-0.2912 , -1.37581],
       [-0.01717, -4.07336],
       [-0.02247, -3.80661],
       [-0.37999, -1.1516 ],
       [-1.04822, -0.43165],
       [-0.5302 , -0.88791],
       [-0.09028, -2.44967],
       [-0.0044 , -5.42932],
       [-0.20785, -1.67305],
       [-0.4499 , -1.01526]], dtype=float32)

In [11]:
np.exp([-0.2912 , -1.37581])

array([0.74737, 0.25263])